---
# Data Mining - Project 3 - Spring Semester 2020
---
##  Iglezou Myrto  | sdi1700038
---

In [69]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import sys # only needed to determine Python version number
import matplotlib # only needed to determine Matplotlib version number
import nltk
from pandas import DataFrame, read_csv
from nltk import word_tokenize, BigramCollocationFinder
from nltk.corpus import stopwords as stopwords2

In [70]:
trainPath = r"C:\Users\myrto\Documents\GitHub\-Project-3-DataMining\train.csv" 
testPath = r"C:\Users\myrto\Documents\GitHub\-Project-3-DataMining\test.csv" 
tempfile = pd.read_csv(trainPath,index_col=False)
traindf = pd.DataFrame(data=tempfile)
tempfile = pd.read_csv(testPath,index_col=False)
testdf = pd.DataFrame(data=tempfile)

In [71]:
from string import punctuation
from wordcloud import STOPWORDS,WordCloud

punctuation = list(punctuation)
punctuation.append('’')
stopwordsPath = r"C:\Users\myrto\Documents\GitHub\-Project-3-DataMining\english_stopwords.txt"
file = open(stopwordsPath,"r",errors='ignore')
stopwords_fromHub = [x.replace('\n',"") for x in file.readlines()]
stopwords = set(STOPWORDS)
stopwords_nltk =  set(stopwords2.words('english'))
stopwords.update(stopwords_nltk)
stopwords.update(set(stopwords_fromHub))

punctuation.append("``")
punctuation.append("...")

In [76]:
import unicodedata
from nltk.stem import WordNetLemmatizer

def normalize_word(word):
    # Will remove the encode token by token
    word = str(word)
    word = unicodedata.normalize('NFKD', word).encode('ASCII','ignore').decode('ASCII')
    return word

def textProcessor(inputData):
    processed_data = []   
    for text in inputData.values:
        textWords = []
        for word in word_tokenize(text):
            # word = normalize_word(word)
            word = word.lower()
            word = WordNetLemmatizer().lemmatize(word)
            word.replace('\n',' ')
            word.replace('<html>','')
            word.replace('<p>','')
            word.replace("'s",'')
            if word.isnumeric():
                word = '#num#'
            if (word not in stopwords) and (word not in punctuation) and word.isalpha():
                textWords.append(word)
        processed_data.append(' '.join(textWords))
        
    inputData.update(pd.Series(processed_data,index=inputData.index))


---
# Naive Bayes
---

In [82]:
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

testdf['Comment'] = testdf['Comment'].str.replace('http\S+|www.\S+', '', case=False)
traindf['Comment'] = traindf['Comment'].str.replace('http\S+|www.\S+', '', case=False)

train = traindf['Comment']
test = testdf['Comment']

textProcessor(train)
textProcessor(test)

countVectorizer = CountVectorizer(ngram_range=(2,2))
count_train = countVectorizer.fit_transform(train)
count_test = countVectorizer.fit_transform(test)

count_train = count_train.toarray()
count_test = count_test.toarray()


# - Gaussian